## Adding a Missing Indicator variable with Scikit-learn ==> MissingIndicator

Scikit-learn provides the **MissingIndicator** class to add a binary variable that flags NA. The MissingIndicator has the option of adding a Missing indicator binary variable to all the variables in the dataset, or only those that show NA in the train set. **Attention!** The transformer only returns the binary variables, which need to be added to the original train data.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.model_selection import train_test_split

**A mix of categorical and numerical variables!**

In [2]:
cols_to_use = ['BsmtQual', 'FireplaceQu', 'MSZoning',
               'BsmtUnfSF', 'LotFrontage', 'MasVnrArea',
               'Street', 'Alley', 'SalePrice']

**Load the House Prices dataset!**

In [3]:
data = pd.read_csv('housingPrices_train.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 9)


MSZoning  LotFrontage Street Alley  MasVnrArea BsmtQual  BsmtUnfSF  \
0       RL         65.0   Pave   NaN       196.0       Gd        150   
1       RL         80.0   Pave   NaN         0.0       Gd        284   
2       RL         68.0   Pave   NaN       162.0       Gd        434   
3       RL         60.0   Pave   NaN         0.0       TA        540   
4       RL         84.0   Pave   NaN       350.0       Gd        490   

  FireplaceQu  SalePrice  
0         NaN     208500  
1          TA     181500  
2          TA     223500  
3          Gd     140000  
4          TA     250000

**Check the null values**

In [4]:
data.isnull().mean()

MSZoning       0.000000
LotFrontage    0.177397
Street         0.000000
Alley          0.937671
MasVnrArea     0.005479
BsmtQual       0.025342
BsmtUnfSF      0.000000
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [5]:
cols_to_use.remove('SalePrice')
X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use], # just the features
                                                    data['SalePrice'], # the target
                                                    test_size=0.3, # the percentage of obs in the test set
                                                    random_state=0) # for reproducibility
X_train.shape, X_test.shape

((1022, 8), (438, 8))

**Check the misssing data again!**

In [6]:
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
MSZoning       0.000000
BsmtUnfSF      0.000000
LotFrontage    0.184932
MasVnrArea     0.004892
Street         0.000000
Alley          0.939335
dtype: float64

## Add a Missing Indicator

In [7]:
indicator = MissingIndicator(error_on_new=True, features='missing-only')
indicator.fit(X_train)  

MissingIndicator()

**See the features with na: the result shows the index!**

In [8]:
indicator.features_

array([0, 1, 4, 5, 7], dtype=int64)

**Find the feature names by passing the index to the list of columns**

In [9]:
X_train.columns[indicator.features_]

Index(['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'Alley'], dtype='object')

**The indicator returns only the additional indicators when we transform the dataset!**

In [10]:
tmp = indicator.transform(X_train)
tmp

array([[False,  True,  True, False,  True],
       [False, False,  True, False,  True],
       [False,  True, False, False,  True],
       ...,
       [ True,  True, False, False,  True],
       [False, False,  True, False,  True],
       [False,  True, False, False,  True]])

**Join it manually to the original X_train! Create a column name for each of the new MissingIndicators!**

In [11]:
indicator_cols = [c+'_NA' for c in X_train.columns[indicator.features_]]
X_train = pd.concat([  # Now concatenate
    X_train.reset_index(),
    pd.DataFrame(tmp, columns = indicator_cols)],
    axis=1)
X_train.head()

index BsmtQual FireplaceQu MSZoning  BsmtUnfSF  LotFrontage  MasVnrArea  \
0     64       Gd         NaN       RL        318          NaN       573.0   
1    682       Gd          Gd       RL        288          NaN         0.0   
2    960       TA         NaN       RL        162         50.0         0.0   
3   1384       TA         NaN       RL        356         60.0         0.0   
4   1100       TA         NaN       RL          0         60.0         0.0   

  Street Alley  BsmtQual_NA  FireplaceQu_NA  LotFrontage_NA  MasVnrArea_NA  \
0   Pave   NaN        False            True            True          False   
1   Pave   NaN        False           False            True          False   
2   Pave   NaN        False            True           False          False   
3   Pave   NaN        False            True           False          False   
4   Pave   NaN        False            True           False          False   

   Alley_NA  
0      True  
1      True  
2      True  
3      True  
4      True

**Now the same for the test set**

In [12]:
tmp = indicator.transform(X_test)
X_test = pd.concat([
    X_test.reset_index(),
    pd.DataFrame(tmp, columns = indicator_cols)],
    axis=1)
X_test.head()

index BsmtQual FireplaceQu MSZoning  BsmtUnfSF  LotFrontage  MasVnrArea  \
0    529       TA          TA       RL        816          NaN         NaN   
1    491       TA          TA       RL        238         79.0         0.0   
2    459       TA          TA       RL        524          NaN       161.0   
3    279       Gd          TA       RL        768         83.0       299.0   
4    655       TA         NaN       RM        525         21.0       381.0   

  Street Alley  BsmtQual_NA  FireplaceQu_NA  LotFrontage_NA  MasVnrArea_NA  \
0   Pave   NaN        False           False            True           True   
1   Pave   NaN        False           False           False          False   
2   Pave   NaN        False           False            True          False   
3   Pave   NaN        False           False           False          False   
4   Pave   NaN        False            True           False          False   

   Alley_NA  
0      True  
1      True  
2      True  
3      True  
4      True

### SimpleImputer on the entire dataset

**Impute the missing values with SimpleImputer! Create an instance! Indicate that we want to impute with the 
most frequent category!**

In [13]:
imputer = SimpleImputer(strategy='most_frequent')
imputer.fit(X_train)  # Fit the imputer to the train set

SimpleImputer(strategy='most_frequent')

**Look at the learnt frequent values like this:**

In [14]:
imputer.statistics_

array([0, 'TA', 'Gd', 'RL', 0, 60.0, 0.0, 'Pave', 'Pave', False, False,
       False, False, True], dtype=object)

**Note** that the transformer learns the most frequent value for both categorical AND numerical variables.

**Impute the train and test set! Data is a numpy array!!!**

In [15]:
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)
X_train

array([[64, 'Gd', 'Gd', ..., True, False, True],
       [682, 'Gd', 'Gd', ..., True, False, True],
       [960, 'TA', 'Gd', ..., False, False, True],
       ...,
       [1216, 'TA', 'Gd', ..., False, False, True],
       [559, 'Gd', 'TA', ..., True, False, True],
       [684, 'Gd', 'Gd', ..., False, False, True]], dtype=object)